<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test runs</a></span></li><li><span><a href="#Interact" data-toc-modified-id="Interact-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Interact</a></span></li><li><span><a href="#Parameter-sweep" data-toc-modified-id="Parameter-sweep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parameter sweep</a></span><ul class="toc-item"><li><span><a href="#Save-the-data" data-toc-modified-id="Save-the-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Save the data</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.cell_models import cell_v1

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, interact_manual

from statsmodels.tsa.stattools import acf

from scipy.signal import find_peaks

In [3]:
from src.analysis import get_cycle_stats

In [4]:
from src.plot_utils import run_and_plot_test

# Test runs

Build something with interact so that you can get a rough sense of how the system behaves. 

In [ ]:
# cell_test, periods= run_and_plot_test()

# Interact

In [5]:
step=.1

In [19]:
interact_manual(
    run_and_plot_test, 
    alpha=(.1, 10., step), 
    beta0=(.001, .01, .001), 
    epsilon=fixed(.2), #(.1, 1., step), 
    gamma=(.01, .05, .01), 
    time_SG2 = fixed(12),#(1, 15), 
    transition_th=fixed(1.), 
    delta=fixed(1.),
    dt=fixed(1e-1), 
    division=fixed("timer"), 
    transition=fixed("size"), 
    T=fixed(4000)
);

interactive(children=(FloatSlider(value=2.0, description='alpha', max=10.0, min=0.1), FloatSlider(value=0.01, …